##### 【 의사결정나무 알고리즘 - DT + CV + Pipeline】
- iris(붓꽃) 품종 분류 모델
- 학습종류 : 지도학습 - 분류
- 학습방법 : DT기반 분류  

[1] 모듈 로딩 및 데이터 준비<hr>

In [22]:
## ==================================================
## [1-1] 모듈 로딩
## ==================================================
import pandas as pd 
import numpy as np

## ML학습 관련
from sklearn.tree import DecisionTreeClassifier

## ML 데이터셋 및 전처리 관련
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder

## ML CV, Pipeline 관련 => 모델 일반화/최적 하이퍼파라미터 조사 및 데이터 누수 해결
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline 

## ML 성능지표 관련
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import classification_report 

## 시각화 관련
import matplotlib.pyplot as plt
import graphviz

In [23]:
## ==================================================
## [1-2] 데이터 준비 및 확인
## ==================================================
## 데이터
DATA_FILE = '../Data/iris.csv'

## 데이터 로딩
irisDF = pd.read_csv(DATA_FILE)

## 데이터 기본정보 확인
display( irisDF.head(2) )
irisDF.info()


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


[2] 학습 준비<hr>

In [24]:
## ==================================================
## [2-1] 피쳐/타겟 분리
## ==================================================
featureDF = irisDF[irisDF.columns[:-1]]
targetSR  = irisDF[irisDF.columns[-1]]

print(f'featureDF:{featureDF.shape},  targetSR:{targetSR.shape}')

featureDF:(150, 4),  targetSR:(150,)


In [25]:
## ==================================================
## [2-2] 학습용/테스트용 분리
## ==================================================
x_train, x_test, y_train, y_test = train_test_split(featureDF,
                                                    targetSR,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=targetSR)

print(f'[TRAIN] x_train:{x_train.shape},  y_train:{y_train.shape}')
print(f'[TEST] x_test:{x_test.shape},  y_test:{y_test.shape}')

[TRAIN] x_train:(120, 4),  y_train:(120,)
[TEST] x_test:(30, 4),  y_test:(30,)


In [26]:
## ==================================================
## [2-3] 타켓 컬럼 인코딩 처리 
## ==================================================
lbEncoder  = LabelEncoder()

en_y_train = lbEncoder.fit_transform(y_train)  ## 학습용 타겟으로 인코더 생성 후 변환까지 진행
en_y_test  = lbEncoder.transform(y_test)


[3] 학습 진행 <hr>

In [27]:
## ===================================================================
## 교차검증을 통한 일반화 성능 체크 + 최적의 하이퍼파라미터 찾기
## => GrdiSearchCV : 분류/회귀 교차검증, 최적 파라미터값, 성능 
## => Pipeline     : 튜닝 시 데이터 누수에 대한 보완 
## ===================================================================
## [3-1] Pipeline 인스턴스 생성 
##       -> 수치형 피쳐 : 스케일링
##       -> 모델 인스턴스 
pipeline = Pipeline(steps=[  ('scaler', RobustScaler()), 
                             ('dt_cls', DecisionTreeClassifier(random_state=42)) ]) 

## [3-2] GridSearchCV 준비
##       -> 최적화할 하이퍼파라미터 값 dict 
##       -> 모델 인스턴스 
##            { "pipeline_모델변수명_ _하이퍼파라미터이름" : [학습알고리즘 클래스 확인]  }
param_dict_ = { "dt_cls__max_depth" : [3, 5, 7], 
                "dt_cls__criterion" : ['gini', 'entropy', 'log_loss']}

## GridSearchCV 인스턴스 생성
gridCV = GridSearchCV(pipeline,
                      param_grid=param_dict_, 
                      return_train_score=True)


In [28]:
## [3-3] 교차검증 통한 최적의 하이퍼라마터 조사 및 성능 평가
gridCV.fit(x_train, en_y_train)

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'dt_cls__criterion': ['gini', 'entropy', ...], 'dt_cls__max_depth': [3, 5, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,with_centering,True


In [29]:
## 학습 후 모델 파라미터 통한 결과 확인 
print(f'best_score_  : {gridCV.best_score_}')
print(f'best_params_ : {gridCV.best_params_}')

best_score_  : 0.9416666666666668
best_params_ : {'dt_cls__criterion': 'gini', 'dt_cls__max_depth': 5}


In [30]:

mean_list =[] 
for i in resultDF.columns.to_list():
    if ('mean' in  i) and ('time' not in i):
        mean_list.append(i)
    elif 'rank' in i:
        mean_list.append(i)
    elif 'param' in i:
        mean_list.append(i)



resultDF = resultDF[mean_list].drop('params',axis=1)

resultDF

,param_dt_cls__criterion,param_dt_cls__max_depth,mean_test_score,rank_test_score,mean_train_score
0,gini,3,0.933333,3,0.981250
1,gini,5,0.941667,1,1.000000
2,gini,7,0.941667,1,1.000000
3,entropy,3,0.925000,6,0.981250
4,entropy,5,0.933333,3,0.997917
5,entropy,7,0.925000,6,1.000000
6,log_loss,3,0.925000,6,0.981250
7,log_loss,5,0.933333,3,0.997917
8,log_loss,7,0.925000,6,1.000000


In [31]:
## 전체 교차 검증별 학습점수, 검증점수 cv_results_
pd.options.mode.copy_on_write=True

resultDF  = pd.DataFrame(gridCV.cv_results_)

print( resultDF.columns )


resultDF  = resultDF[[
                      'mean_test_accuracy', 
                      'mean_test_f1_micro',
                      'mean_test_precision_micro', 
                      'mean_test_recall_micro',

                      'mean_train_accuracy', 
                      'mean_train_f1_micro',
                      'mean_train_precision_micro', 
                      'mean_train_recall_micro',

                       'param_dt_cls__max_depth', 
                       'param_dt_cls__criterion']]


resultDF.rename(columns={'param_dt_cls__max_depth':'max_depth', 
                         'param_dt_cls__criterion':'criterion',
                         'rank_test_score':'rank_test'}, inplace=True)



Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_dt_cls__criterion', 'param_dt_cls__max_depth', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'split0_train_score',
       'split1_train_score', 'split2_train_score', 'split3_train_score',
       'split4_train_score', 'mean_train_score', 'std_train_score'],
      dtype='object')


KeyError: "['mean_test_accuracy', 'mean_test_f1_micro', 'mean_test_precision_micro', 'mean_test_recall_micro', 'mean_train_accuracy', 'mean_train_f1_micro', 'mean_train_precision_micro', 'mean_train_recall_micro'] not in index"

[4] 다양한 성능평가 및 최종학습 시 성능평가 설정 <hr>

In [ ]:
## ===================================================================
## 교차검증을 통한 일반화 성능 체크 + 최적의 하이퍼파라미터 찾기
## => GrdiSearchCV : 분류/회귀 교차검증, 최적 파라미터값, 성능 
## => Pipeline     : 튜닝 시 데이터 누수에 대한 보완 
## ===================================================================
## [4-1] Pipeline 인스턴스 생성 
##       -> 수치형 피쳐 : 스케일링
##       -> 모델 인스턴스 
pipeline = Pipeline(steps=[  ('scaler', RobustScaler()), 
                             ('dt_cls', DecisionTreeClassifier(random_state=42)) ]) 

## [4-2] GridSearchCV 준비
##       -> 최적화할 하이퍼파라미터 값 dict 
##       -> 모델 인스턴스 
##            { "pipeline_모델변수명_ _하이퍼파라미터이름" : [학습알고리즘 클래스 확인]  }
param_dict_ = { "dt_cls__max_depth" : [3, 5, 7], 
                "dt_cls__criterion" : ['gini', 'entropy', 'log_loss']}

## 여러 개 scoring 설정 -----------------------------------------
##  - accuracy: 정확도
##  - f1_micro: 클래스별 F1 평균(불균형일수록 macro/weighted가 중요)
##  - recall_micro, precision_micro 같이 추가 가능
scoring = {
    "accuracy": "accuracy",
    "f1_micro": "f1_micro",
    "precision_micro": "precision_micro",
    "recall_micro": "recall_micro"
}

## GridSearchCV 인스턴스 생성
## -> refit='f1_micro' : 가장 좋은 파라미터 조합으로 최종 학습 시
##                       성능평가 기준 설정 
gridCV = GridSearchCV(pipeline,
                      param_grid=param_dict_, 
                      return_train_score=True,
                      scoring=scoring,
                      refit='f1_micro')


In [ ]:
## [4-3] 교차검증 통한 최적의 하이퍼라마터 조사 및 성능 평가
gridCV.fit(x_train, en_y_train)

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'dt_cls__criterion': ['gini', 'entropy', ...], 'dt_cls__max_depth': [3, 5, ...]}"
,scoring,"{'accuracy': 'accuracy', 'f1_micro': 'f1_micro', 'precision_micro': 'precision_micro', 'recall_micro': 'recall_micro'}"
,n_jobs,None
,refit,'f1_micro'
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,with_centering,True


In [ ]:
## 학습 후 모델 파라미터 통한 결과 확인 
print(f'best_score_  : {gridCV.best_score_}')
print(f'best_params_ : {gridCV.best_params_}')

best_score_  : 0.9416666666666668
best_params_ : {'dt_cls__criterion': 'gini', 'dt_cls__max_depth': 5}


In [ ]:
## 전체 교차 검증별 학습점수, 검증점수 cv_results_
pd.options.mode.copy_on_write=True

resultDF  = pd.DataFrame(gridCV.cv_results_)

print( resultDF.columns )

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_dt_cls__criterion', 'param_dt_cls__max_depth', 'params',
       'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy',
       'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy',
       'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy',
       'split1_train_accuracy', 'split2_train_accuracy',
       'split3_train_accuracy', 'split4_train_accuracy', 'mean_train_accuracy',
       'std_train_accuracy', 'split0_test_f1_micro', 'split1_test_f1_micro',
       'split2_test_f1_micro', 'split3_test_f1_micro', 'split4_test_f1_micro',
       'mean_test_f1_micro', 'std_test_f1_micro', 'rank_test_f1_micro',
       'split0_train_f1_micro', 'split1_train_f1_micro',
       'split2_train_f1_micro', 'split3_train_f1_micro',
       'split4_train_f1_micro', 'mean_train_f1_micro', 'std_train_f1_micro',
       'split0_test_precision_micro', 'split1_test_precision_micr